# Downlaod Live ATC

Doesn't work yet (problem with ffmep)

TODO: Retry later as ffmep has been installed in ~/.bashrc (source ~/.bashrc)

In [ ]:
import subprocess
import datetime
import os

In [ ]:
# Stream URL from LiveATC.net (must be manually found or provided by LiveATC)
stream_url = "http://d.liveatc.net/epwa_app"  # Example: Warsaw Approach

# Set duration of recording in seconds
duration = 60 * 5  # 5 minutes

# Output filename
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"ksfo_twr_{timestamp}.mp3"

# Use ffmpeg to record stream
subprocess.run([
    "ffmpeg",
    "-i", stream_url,
    "-t", str(duration),
    "-acodec", "libmp3lame",
    filename
])

In [ ]:
ffmpeg_path = os.path.expanduser("/mnt/c/Users/kruu/tools/ffmpeg/ffmpeg")
stream_url = "http://d.liveatc.net/epwa_app"
duration = 300  # seconds

# Output file
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"epwa_app_{timestamp}.mp3"

# Run recording
subprocess.run([
    ffmpeg_path,
    "-re",
    "-i", stream_url,
    "-t", str(duration),
    "-acodec", "libmp3lame",
    output_file
])

# Download Recordings

In [ ]:
import os
import re
import requests
import urllib.request
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin
from datetime import datetime, timedelta


In [2]:
def get_stations(icao):
  page = requests.get(f'https://www.liveatc.net/search/?icao={icao}')
  soup = BeautifulSoup(page.content, 'html.parser')

  stations = soup.find_all('table', class_='body', border='0', padding=lambda x: x != '0')
  freqs = soup.find_all('table', class_='freqTable', colspan='2')

  for table, freqs in zip(stations, freqs):
    title = table.find('strong').text
    up = table.find('font').text == 'UP'
    href = table.find('a', href=lambda x: x and x.startswith('/archive.php')).attrs['href']

    identifier = re.findall(r'/archive.php\?m=([a-zA-Z0-9_]+)', href)[0]

    frequencies = []
    rows = freqs.find_all('tr')[1:]
    for row in rows:
      cols = row.find_all('td')
      freq_title = cols[0].text
      freq_frequency = cols[1].text

      frequencies.append({'title': freq_title, 'frequency': freq_frequency})
      
    yield {'identifier': identifier, 'title': title, 'frequencies': frequencies, 'up': up}


In [3]:
stations = get_stations("epwa")

for station in stations:
    print(f"[{station['identifier']}] - {station['title']}")

    for freq in station['frequencies']:
      print(f"\t{freq['title']} - {freq['frequency']}")

[epwa_app] - EPWA Approach #1
	Warsaw Approach - 128.805
[epwa_app2] - EPWA Approach #2
	Warsaw Approach - 125.055
[epwa_gnd] - EPWA Ground
	Warsaw Ground - 121.900
[epwa_twr2] - EPWA Tower
	Warsaw Tower - 118.300


In [43]:
def get_last_zulu_period(date, minutes=30):
  return date - timedelta(minutes=minutes) - (date - datetime.min) % timedelta(minutes=minutes)

In [12]:
def download_archive(station, date, time, save_path):
  page = requests.get(f'https://www.liveatc.net/archive.php?m={station}')
  soup = BeautifulSoup(page.content, 'html.parser')
  archive_identifer = soup.find('option', selected=True).attrs['value']

  filename = f'{archive_identifer}-{date}-{time}.mp3'

  path = f'/tmp/{filename}'
  url = f'https://archive.liveatc.net/{station.split("_")[0]}/{filename}'
  print(url)
  
  # Use headers to simulate a browser
  headers = {'User-Agent': 'Mozilla/5.0'}
  
  # Download with requests
  response = requests.get(url, headers=headers, stream=True)
  
  # Check if the file exists (200 OK)
  if response.status_code == 200:
      os.makedirs(save_path, exist_ok=True)
      path = os.path.join(save_path, filename)
      with open(path, 'wb') as f:
          for chunk in response.iter_content(chunk_size=8192):
              f.write(chunk)
      print("Download successful:", path)
  else:
      print("Download failed:", response.status_code, response.reason)

In [13]:
download_archive('epwa_app', 'Jun-15-2025', '1000Z', "data_EPWA")

https://archive.liveatc.net/epwa/EPWA-App-Jun-15-2025-1000Z.mp3
Download successful: data_EPWA/EPWA-App-Jun-15-2025-1000Z.mp3
